#  Consignes

## Description

Ouvrir le fichier ks-projects-201801.csv, il recense environ 100 000 projets KickStarter. Intégrer les données directement avec L'API Python dans une base de données Mongo. 

Il conviendra de bien spécifier manuellement l'ID du document. Pensez aussi à bien formatter le type des données pour profiter des méthodes implémentées par Mongo. L'ensemble de données n'est pas forcément nécessaire, c'est à vous de créer votre modèle de données.

## Questions

- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
- 2) Compter le nombre de projets ayant atteint leur but.
- 3) Compter le nombre de projets pour chaque catégorie.
- 4) Compter le nombre de projets français ayant été instanciés avant 2016.
- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
- 6) Compter le nombre de projet ayant "Sport" dans leur nom

In [15]:
import pandas as pd
import pymongo
from pymongo import MongoClient

In [16]:
client = MongoClient("mongo")
database = client['exercices']
collection = database['kickstarter']

In [17]:
df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv")
df_ks.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000,2011-08-17 06:31:31,1145,canceled,24,US,1145,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000,2012-07-23 20:46:48,1851,failed,28,US,1851,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000,2016-07-01 21:55:54,7534,successful,254,US,3796,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000,2017-08-28 15:47:02,16298,successful,367,US,2670,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000,2016-01-25 17:37:10,1,failed,1,CA,0.708148,0.738225


Ce warning intervient lorsque pandas n'arrive pas à inférer le type de données. Il est sympa il précise les colones 6,8,10,12. 

In [18]:
df_ks.columns[[6,8,10,12]]

Index(['goal', 'pledged', 'backers', 'usd pledged'], dtype='object')

## Question 0

### Netoyer les données

In [96]:
collection.drop()

### Importer les données

In [97]:
l_d = df_ks.to_dict(orient='records')
collection.insert_many(l_d)

## Question 1  

In [98]:
cur = collection.find().sort([("pledged", -1)]).limit(5)
list(cur)

[{'_id': ObjectId('602e5142368c0c0091ea4d86'),
  'ID': 218218259,
  'name': 'True North ï¿½ Repurposed Sailcloth Products',
  'category': 'Product Design',
  'main_category': 'Design',
  'currency': 'USD',
  'deadline': '2016-06-14',
  'goal': '7500.0',
  'launched': '2016-05-10 15:53:56',
  'pledged': '9996.0',
  'state': 'successful',
  'backers': '50',
  'country': 'US',
  'usd pledged': '9996.0',
  'usd_pledged_real': 9996.0},
 {'_id': ObjectId('602e5142368c0c0091e9fc64'),
  'ID': 1712352341,
  'name': 'Scott Davis - Solo Record',
  'category': 'Music',
  'main_category': 'Music',
  'currency': 'USD',
  'deadline': '2014-11-22',
  'goal': '8000.0',
  'launched': '2014-10-23 20:06:37',
  'pledged': '9995.0',
  'state': 'undefined',
  'backers': '0',
  'country': 'N,0"',
  'usd pledged': nan,
  'usd_pledged_real': 9995.0},
 {'_id': ObjectId('602e5142368c0c0091e9d414'),
  'ID': 1742823215,
  'name': 'Polco: A Digital Platform for Better Civic Communication',
  'category': 'Web',
  'ma

## Question 2

In [99]:
cur = collection.find({"state":"successful"}).count()
cur

53040

## Question 3

In [100]:
cur = collection.aggregate([{"$group" : {"_id" : "$category", "projectNumberByCategory" : {"$sum" : 1}}}])
list(cur)

[{'_id': 'Small Batch', 'projectNumberByCategory': 701},
 {'_id': 'Narrative Film', 'projectNumberByCategory': 2099},
 {'_id': 'Fashion', 'projectNumberByCategory': 3379},
 {'_id': 'Experimental', 'projectNumberByCategory': 357},
 {'_id': 'Jewelry', 'projectNumberByCategory': 472},
 {'_id': 'Product Design', 'projectNumberByCategory': 8886},
 {'_id': 'Hardware', 'projectNumberByCategory': 1431},
 {'_id': 'Webcomics', 'projectNumberByCategory': 259},
 {'_id': 'Food Trucks', 'projectNumberByCategory': 687},
 {'_id': 'Romance', 'projectNumberByCategory': 74},
 {'_id': 'R&B', 'projectNumberByCategory': 172},
 {'_id': 'Nonfiction', 'projectNumberByCategory': 3390},
 {'_id': '3D Printing', 'projectNumberByCategory': 271},
 {'_id': 'Young Adult', 'projectNumberByCategory': 328},
 {'_id': 'Embroidery', 'projectNumberByCategory': 49},
 {'_id': 'Community Gardens', 'projectNumberByCategory': 115},
 {'_id': 'Punk', 'projectNumberByCategory': 127},
 {'_id': 'Video Art', 'projectNumberByCategory': 

## Question 4

In [101]:
cur = collection.find({"country":"FR", "launched":{"$lt" : "2016-01-01"}}).count()
cur

330

## Question 5

In [102]:
cur = collection.find({"country":"US", "usd_pledged_real":{"$gt" : 200000}}).count()
cur

658

## Question 6 

In [104]:
cur = collection.find({"name":{'$regex':'Sport'}}).count()
cur

323